## The code below,

1. Identifies all the studyData's survey response recordings
2. Iterates over all the responses and converts each of its recordings from .webm to .mp4 video file format
3. Identifies all the responses that are repeated for a particular stimulusVideo

In [ ]:
import os
import re
import glob
import subprocess
import pandas as pd

from tqdm import tqdm

###### Pilot studyUsers and data path

In [ ]:
pilot_studyDataDirectory = '../../../code_base/pilot_3/collected_data/study_videos/'
# participantLogDirectory = '/Users/sukruthgl/Desktop/Spring/CS 5999 - MEng Project/StudyData/pilots/participant_log.xlsx'

participantIDPattern = r'^[0-9]{4}$'

pilot_studyUsers = os.listdir(pilot_studyDataDirectory)
pilot_studyUsers = sorted([user for user in pilot_studyUsers if re.match(participantIDPattern, user)])
pilot_studyUsers

###### New studyUsers to be processed as and when the survey is completed

In [ ]:
newStudyUserDirectory = '../../../code_base/pilot_3/collected_data/study_videos/'
newStudyUsers = os.listdir(newStudyUserDirectory)
newStudyUsers = [user for user in newStudyUsers if re.match(participantIDPattern, user)]

newSurveyers = []

for user in newStudyUsers:
    if user not in pilot_studyUsers:
        newSurveyers.append(user)
        
print(newSurveyers)
print(len(newSurveyers))

##### The below code performs:

`.webm` to `.mp4` video conversion

It takes in 
- a list of `studyUsers` consists of the `randomID` of the participants during the qualtrics survey. This ID also acts as the directory name under which the participant recordings are created and stored.
- `studyDataDirectory` : which is the path at which the studyData recordings are stored at

In [ ]:
# Iterates through all the studyUserData directories 
# and then creates a new directory and stores all the converted (".webm" to ".mp4") files in it

def convertVideo(studyUsers, studyDataDirectory):
    
    for studyUser in tqdm(studyUsers, desc='Study Participants Processed: '):
        studyUserDirectory = studyDataDirectory + studyUser

        # Creating a new directory to store the converted video files
        studyUser_mp4Directory = studyUserDirectory + '/mp4StudyVideo'
        
        if not os.path.exists(studyUser_mp4Directory):
            os.mkdir(studyUser_mp4Directory)

        for video in glob.glob(f"{studyUserDirectory}/*.webm"):
            output_video = os.path.join(studyUser_mp4Directory, os.path.basename(video).replace(".webm", ".mp4"))
            subprocess.run(["ffmpeg", "-i", video, "-r", "30", output_video], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

In [ ]:
convertVideo(pilot_studyUsers, pilot_studyDataDirectory)

###### The below method identifies all the study recordings that have been generated more than once for a particular stimulus video

In [ ]:
# NOTE:  Need to rewrite this to make it more efficient,
# by appending the repeated recordings directly to the participant_log_df
def repeatedRecordings(studyUsers, studyDataDirectory):
    for studyUser in studyUsers:

        studyUserDirectory = studyDataDirectory + f'/{studyUser}'
        studyUser_mp4Directory = studyUserDirectory + f'/mp4StudyVideo'

    #     Obtaining all the recordings of the responses
        recordings = os.listdir(studyUser_mp4Directory)

    #     Splitting the responses to include only the stimulusVideoNumber
        recording_IDs = []
        for recording in recordings:
            recording_IDs.append(recording.split("_")[0])

    #     Creating a dict to identify the stimulusVideoNumbers and their count of views by the users
        repeatedRecordings = {}
        for recording_ID in recording_IDs:
            if recording_ID in repeatedRecordings:
                repeatedRecordings[recording_ID] += 1
            else:
                repeatedRecordings[recording_ID] = 1

    #     Identifying the stimulusVideos for which there are multiple views by the users
        repeatedResponses = []    
        for recordingID, views in repeatedRecordings.items():
            if (views >= 2):
                repeatedResponses.append(recordingID)

        repeatedResponses_string = ', '.join(repeatedResponses)
        print(f'{studyUser}', repeatedResponses_string)

    #     repeatedResponses = studyUser, [recordingID for recordingID, views in repeatedRecordings.items() if (views >= 2)]
    # #     repeatedResponses = {recordingID: views for recordingID, views in repeatedRecordings.items() if (views >= 2)}
    #     print(repeatedResponses)



In [ ]:
repeatedRecordings(pilot_studyUsers, pilot_studyDataDirectory)

In [ ]:
# participant_log_df = pd.read_excel(participantLogDirectory)
# participant_log_df